In [44]:
import sys
import polars as pl
import boto3
import time
from IPython.display import display
import datetime
import math
from pathlib import PurePosixPath
print(f"{sys.version=}")

sys.version='3.14.0 (main, Oct  7 2025, 09:34:52) [Clang 17.0.0 (clang-1700.3.19.1)]'


# Список silver S3

In [2]:
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)
storage_options = {
    "aws_endpoint_url": "http://localhost:9000", # Важно: aws_endpoint_url (иногда endpoint_url)
    "aws_access_key_id": "minioadmin",
    "aws_secret_access_key": "minioadmin",
    "aws_region": "us-east-1", # Для MinIO часто можно оставить us-east-1
    "aws_allow_http": "true",  # Разрешить HTTP (без SSL)
}

In [74]:
bucket = s3.Bucket('silver')

In [79]:
def silver_path2dt(obj):
    return datetime.datetime(**{item.split("=",1)[0]: int(item.split("=",1)[1]) for item in PurePosixPath(obj.key).parts if "=" in item})
objects = sorted(bucket.objects.all(), key=silver_path2dt)

In [76]:
objects[0], objects[1]

(s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=12/part-e9274a27-ad7e-4a4d-a85b-e9f36b1d416b.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=13/part-a4272eab-a741-4854-89bf-54ddd1041b10.parquet'))

In [77]:
for obj in objects:
    key = obj.key
    size_mb = obj.size//1e+6
    print(key, size_mb)

hh/vacancies/year=2025/month=11/day=12/part-e9274a27-ad7e-4a4d-a85b-e9f36b1d416b.parquet 5.0
hh/vacancies/year=2025/month=11/day=13/part-a4272eab-a741-4854-89bf-54ddd1041b10.parquet 5.0
hh/vacancies/year=2025/month=11/day=14/part-efb2c48a-8da8-4f54-98c6-51b0dc823d7f.parquet 4.0
hh/vacancies/year=2025/month=11/day=15/part-ca7eaa96-d306-4c05-a950-ce889c955ef1.parquet 1.0
hh/vacancies/year=2025/month=11/day=16/part-0bfac3be-ede9-4bdd-9077-4dca352c2011.parquet 0.0
hh/vacancies/year=2025/month=11/day=17/part-70b7d214-3e3e-4691-8dc0-33b2ff7612f9.parquet 7.0
hh/vacancies/year=2025/month=11/day=18/part-3a613aca-be01-44ba-900c-6022a6989fc6.parquet 6.0
hh/vacancies/year=2025/month=11/day=19/part-6461b2e2-5853-4d09-8e78-0af8ac4b5bbb.parquet 6.0
hh/vacancies/year=2025/month=11/day=20/part-ff942caa-4915-4b48-8719-d864899b4ea3.parquet 6.0
hh/vacancies/year=2025/month=11/day=21/part-a28303bd-d622-4bd7-ba82-c5e8e0a20c2f.parquet 6.0
hh/vacancies/year=2025/month=11/day=22/part-77997a61-4be0-497d-bb0c-7e

In [80]:
objects = objects[-9:]
objects

[s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=1/part-02be6b9d-137d-4d1b-b5ab-e1b0d5af3a72.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=2/part-dd81b375-bac3-45de-a1ad-9262970ac56e.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=3/part-ae98c7ac-2ed7-48f1-946f-7a348e97dddc.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=4/part-f4b0be4e-ff8a-478a-af1b-b3346bf22235.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=5/part-bed604b5-02b3-4b16-8351-00d48ce04aa6.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=6/part-04dad514-23db-44ac-b676-ad8f170b0423.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=7/part-5a5fe775-6081-4466-a65c-be379d86eee8.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/

# Общее кол-во вакансий в S3

In [81]:
for obj in objects:
    schema = pl.scan_parquet(f"s3://silver/{obj.key}",storage_options=storage_options).collect_schema()
    print(obj, len(schema))

s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=1/part-02be6b9d-137d-4d1b-b5ab-e1b0d5af3a72.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=2/part-dd81b375-bac3-45de-a1ad-9262970ac56e.parquet') 52
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=3/part-ae98c7ac-2ed7-48f1-946f-7a348e97dddc.parquet') 52
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=4/part-f4b0be4e-ff8a-478a-af1b-b3346bf22235.parquet') 52
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=5/part-bed604b5-02b3-4b16-8351-00d48ce04aa6.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=6/part-04dad514-23db-44ac-b676-ad8f170b0423.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=7/part-5a5fe775-6081-4466-a65c-be379d86eee8.parquet') 52
s3.ObjectSummary(bucket_name='silver', ke

In [82]:
df_lazy = pl.scan_parquet(
    [f"s3://silver/{obj.key}" for obj in objects],
    storage_options=storage_options,
)

In [83]:
df_lazy.select(
    pl.col('id').count().alias('id_cnt'),
    pl.col('id').unique().count().alias('unique_id_cnt')
).collect()

SchemaError: extra column in file outside of expected schema: immediate_redirect_url, hint: specify this column in the schema, or pass extra_columns='ignore' in scan options. File containing extra column: 's3://silver/hh/vacancies/year=2025/month=12/day=2/part-dd81b375-bac3-45de-a1ad-9262970ac56e.parquet'.

In [53]:
s3path = f"s3://silver/{obj.key}"

In [56]:
s3path = "s3://silver/hh/vacancies/year=2025/month=11/day=30/part-dd8af1b2-1ae5-4c28-a27e-f5910bbdb0b9.parquet"

In [57]:
df_lazy = pl.scan_parquet(
    s3path,
    storage_options=storage_options,
)

In [58]:
df_lazy.select(
    pl.col('id').count().alias('id_cnt'),
    pl.col('id').unique().count().alias('unique_id_cnt')
).collect()

id_cnt,unique_id_cnt
u32,u32
3878,3878
